In [9]:
import tensorflow as tf
import numpy as np
import pandas

np.set_printoptions(precision=3, suppress=True)

print(tf.__version__)

tf.random.set_seed(777)

2.13.0


In [15]:
xy = np.loadtxt('C:\data-04-zoo.csv', delimiter=',', dtype=np.float32)
#xy = pandas.read_csv(
 #   "https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data", delimiter=',', dtype=np.float32,
  #  names=["animal name", "hair", "feathers", "eggs", "milk", "airborne", "aquatic", "predator", "toothed", "backbone",
   #        "breathes", "venomous", "fins", "legs", "tail", "domestic", "catsize", "type"])
x_data = xy[:, 0:-1] 
y_data = xy[:, -1]

nb_classes = 7  # 0 ~ 6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


y의 0에서 6까지의 데이터를 one hot으로 표시해주기 위해 예를들면
([0], [3]) 즉 shape(2,1) 의 데이터를 [[[1,0,0,0,0,0,0]],[[0,0,0,1,0,0,0]]]의 shape (2,1,7)으로 나타낸후 차원을 (2,7)로 다시 맞춰주기 위해 reshape 함수를 이용한다.

In [19]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X)) #정확도를 맞추기 위해서 넣었음

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, 
                                                      from_logits=True)    
    cost = tf.reduce_mean(cost_i)    
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

가설의 정확도를 알기 위하여 prediction을 이용한다.
pred에서 sotfmax 함수를 통과한 값중 최고를 argmax 함수로 찾은 후 이를 실제값과 비교한다. 이를 평균을 내어 정확도를 계산한다.

In [18]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
#             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 5.610101222991943, Acc: 0.1683168262243271
Steps: 100 Loss: 0.6019306182861328, Acc: 0.8415841460227966
Steps: 200 Loss: 0.38190874457359314, Acc: 0.9108911156654358
Steps: 300 Loss: 0.2877010405063629, Acc: 0.9405940771102905
Steps: 400 Loss: 0.23101474344730377, Acc: 0.9504950642585754
Steps: 500 Loss: 0.19246311485767365, Acc: 0.9603960514068604
Steps: 600 Loss: 0.16442006826400757, Acc: 0.9603960514068604
Steps: 700 Loss: 0.14311201870441437, Acc: 0.9603960514068604
Steps: 800 Loss: 0.12642519176006317, Acc: 0.9603960514068604
Steps: 900 Loss: 0.11306189745664597, Acc: 0.9900990128517151
Steps: 1000 Loss: 0.10216663777828217, Acc: 0.9900990128517151


정확도가 거의 100프로까지 올라가는 것을 볼 수 있다.